In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/jaimergp/autodocktools-prepare-py3k.git


Cloning into 'autodocktools-prepare-py3k'...
remote: Enumerating objects: 468, done.
remote: Total 468 (delta 0), reused 0 (delta 0), pack-reused 468 (from 1)
Receiving objects: 100% (468/468), 1.45 MiB | 7.24 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [ ]:
import os
import subprocess

#-------------------------------- RUTAS -----------------------------------------------#
DRIVE_ROOT          = "/content/drive/MyDrive/TFM/T2/TOT_ligandos_sitios/TOTAL/TOTAL_PDB"
REPO_ROOT           = "/content/autodocktools-prepare-py3k"
ligand_list_file    = os.path.join(DRIVE_ROOT, "lista_ligandos.txt")
INPUT_DIR           = os.path.join(DRIVE_ROOT, "ligandos_pdb")
OUTPUT_DIR          = os.path.join(DRIVE_ROOT, "ligandos_pdbqt")
SCRIPT              = os.path.join(REPO_ROOT,
                                   "AutoDockTools/Utilities24/prepare_ligand4.py")
#--------------------------------------------#

os.makedirs(OUTPUT_DIR, exist_ok=True)

env = os.environ.copy()
env["PYTHONPATH"] = REPO_ROOT

with open(ligand_list_file, 'r') as f:
    for line in f:
        name = line.strip()
        if not name:
            continue

        if not name.lower().endswith(".pdb"):
            name += ".pdb"
        base = os.path.splitext(name)[0]
        out_path = os.path.join(OUTPUT_DIR, f"{base}.pdbqt")


        if os.path.exists(out_path):
            #print(f"{base}.pdbqt ya existe, se omite.")
            continue

        print(f"Convirtiendo {name} y añadiendo hidrógenos…")
        cmd = [
            "python3", SCRIPT,
            "-l", name,
            "-A", "hydrogens",
            "-o", out_path
        ]

        result = subprocess.run(
            cmd,
            cwd=INPUT_DIR,
            env=env,
            capture_output=True,
            text=True
        )
        if result.returncode != 0:
            print(f"Error en {name}:")
            print(result.stderr.strip() or result.stdout.strip())
        else:
            print(f"{base}.pdbqt generado correctamente.")


In [ ]:
import os, subprocess

#----- Ajusta estas rutas a tu entorno -----#
DRIVE_ROOT    = "/content/drive/MyDrive/TFM/T2/TOT_ligandos_sitios/TOTAL/TOTAL_PDB"
REPO_ROOT     = "/content/autodocktools-prepare-py3k"
receptor_list = os.path.join(DRIVE_ROOT, "lista_receptores.txt")
INPUT_DIR     = os.path.join(DRIVE_ROOT, "sitios_pdb")
OUTPUT_DIR    = os.path.join(DRIVE_ROOT, "sitios_pdbqt")
SCRIPT        = os.path.join(REPO_ROOT, "AutoDockTools/Utilities24/prepare_receptor4.py")
#-------------------------------------------#

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Prepara entorno para que Python encuentre MolKit
env = os.environ.copy()
env["PYTHONPATH"] = REPO_ROOT

with open(receptor_list) as f:
    for line in f:
        name = line.strip()
        if not name:
            continue
        if not name.endswith(".pdb"):
            name += ".pdb"
        base = os.path.splitext(name)[0]
        out_path = os.path.join(OUTPUT_DIR, f"{base}.pdbqt")

        # Si ya existe, lo omitimos
        if os.path.exists(out_path):
            print(f"{base}.pdbqt ya existe, se omite.")
            continue

        print(f"Convirtiendo {name} y añadiendo hidrógenos…")
        cmd = [
            "python3", SCRIPT,
            "-r", name,
            "-A", "hydrogens",
            "-o", out_path
        ]

        try:
            # Ejecuta sin check=True para no lanzar excepción
            result = subprocess.run(
                cmd,
                cwd=INPUT_DIR,
                env=env,
                capture_output=True,
                text=True
            )
        except Exception as e:
            print(f"Excepción al invocar el script para {name}: {e}")
            # Continuamos con el siguiente receptor
            continue

        if result.returncode != 0:
            # Reportamos el error pero no detenemos el bucle
            print(f"Error al procesar {name}:")
            #print(result.stderr.strip() or result.stdout.strip())
        else:
            print(f"{base}.pdbqt creado con H correctamente.")
